In [1]:
from PIL import Image

In [2]:
from torchvision import transforms, datasets

In [3]:
imsize = 256
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [4]:
dataset = datasets.ImageFolder('animal_images', transform)
dataset

Dataset ImageFolder
    Number of datapoints: 120
    Root location: animal_images
    StandardTransform
Transform: Compose(
               Resize(size=(128, 128), interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [5]:
import matplotlib.pyplot as plt

In [6]:
n_train = int(len(dataset) * 0.7)
n_val = len(dataset) - n_train 

In [7]:
import torch
# import torchvision
# import torchvision.transforms as transforms

In [8]:
train, val = torch.utils.data.random_split(dataset, [n_train, n_val])

In [9]:
# import torch.nn as nn


import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [10]:
from cnn_model import Net

In [11]:
len(train), len(val)

(84, 36)

In [12]:
# バッチサイズ
batch_size = 32
num_workers = 0

train_loader = torch.utils.data.DataLoader(train, batch_size, num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size, num_workers=num_workers)

trainer = pl.Trainer(max_epochs=10)
model = Net()

trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | conv | Conv2d      | 84    
1 | pool | MaxPool2d   | 0     
2 | bn   | BatchNorm1d | 24 K  
3 | fc1  | Linear      | 1 M   
4 | fc2  | Linear      | 404   
Epoch 0:  60%|██████    | 3/5 [00:03<00:02,  1.02s/it, loss=5.301, v_num=10]
Validating: 0it [00:00, ?it/s]
Epoch 1:  60%|██████    | 3/5 [00:03<00:02,  1.06s/it, loss=5.371, v_num=10]
Validating: 0it [00:00, ?it/s]Saving latest checkpoint..
Epoch 1:  60%|██████    | 3/5 [00:04<00:02,  1.36s/it, loss=5.371, v_num=10]


1

In [13]:
trainer.callback_metrics

{'val_loss': tensor(17.9251),
 'val_acc': tensor(0.1875),
 'loss': tensor(6.9409)}

In [17]:
import torch.nn.functional as F

In [19]:
model.eval()
model.freeze()



In [21]:
ret = F.softmax(model(dataset[60][0].unsqueeze(0)))
print(ret)
ret.argmax(), dataset[60][0]

tensor([[1.4011e-01, 1.3710e-18, 5.4827e-08, 8.5989e-01]])


(tensor(3),
 tensor([[[0.2549, 0.2549, 0.2431,  ..., 0.1961, 0.2235, 0.2353],
          [0.2588, 0.2549, 0.2392,  ..., 0.2078, 0.2078, 0.2000],
          [0.2471, 0.2471, 0.2275,  ..., 0.1804, 0.2000, 0.2157],
          ...,
          [0.1529, 0.1765, 0.1686,  ..., 0.3647, 0.3686, 0.4078],
          [0.1686, 0.2039, 0.1686,  ..., 0.3569, 0.3569, 0.4275],
          [0.2392, 0.2431, 0.2039,  ..., 0.3843, 0.4275, 0.4431]],
 
         [[0.5059, 0.5059, 0.4980,  ..., 0.3686, 0.4000, 0.4078],
          [0.5098, 0.5059, 0.4902,  ..., 0.3725, 0.3843, 0.3922],
          [0.4980, 0.4941, 0.4745,  ..., 0.3451, 0.3725, 0.4039],
          ...,
          [0.3686, 0.3843, 0.4039,  ..., 0.6196, 0.6235, 0.6353],
          [0.3725, 0.4078, 0.4000,  ..., 0.6000, 0.5961, 0.6392],
          [0.4431, 0.4314, 0.3961,  ..., 0.5882, 0.6078, 0.6314]],
 
         [[0.1765, 0.1765, 0.1608,  ..., 0.1490, 0.1725, 0.1765],
          [0.1804, 0.1765, 0.1569,  ..., 0.1529, 0.1569, 0.1569],
          [0.1686, 0.1686, 0

In [43]:
torch.save(model.to('cpu').state_dict(), 'trained.pth')